In [1]:
import tensorflow as tf 

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
lr = 0.001 #learning rate
training_iters = 100000
batch_size = 128

In [6]:
n_inputs = 28 # 28 pixel in each line
n_steps = 28 # 28 lines
n_hidden_units = 128
n_classes = 10 #0-9

In [7]:
x = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,[None,n_classes])

In [8]:
weights = {
    #(28,128)
    'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    #(128,10)
    'out': tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}

In [10]:
biases = {
    # (128,)
    'in':tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),
    # 10,
    'out':tf.Variable(tf.constant(0.1,shape=[n_classes,]))
}

In [45]:
def RNN(X,weights,biases):
    #hidden layer for input to cell
    # X (128batch,28steps,28inputs) => (128*28,28inputs)
    X = tf.reshape(X,[-1,n_inputs])
    # X_in->(128batch,28steps,128hidden)
    X_in = tf.matmul(X,weights['in'])+biases['in']
    # X_in->(128batch,28steps,128hidden)
    X_in = tf.reshape(X_in,[-1,n_steps,n_hidden_units])
    #cell
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units,reuse=True)
     # lstm cell is divided into two parts (c_state, h_state)
    init_state = lstm_cell.zero_state(batch_size,dtype=tf.float32)
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=init_state,time_major=False)
    
    #hidden layer for output as the final resuklts
    # results = tf.matmul(final_state[1], weights['out']) + biases['out']
    
    # # or  bescause :final_state[1]==outputs[-1]
    # unpack to list [(batch, outputs)..] * steps
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']    # shape = (128, 10)  (batch_size,n_classes)
    
    return results

In [46]:
pred = RNN(x, weights, biases)

In [50]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_step = tf.train.AdamOptimizer(lr).minimize(cost)

In [48]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [51]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_step], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
            x: batch_xs,
            y: batch_ys,
            }))
        step += 1


0.203125
0.640625
0.804688
0.765625
0.828125
0.875
0.914063
0.914063
0.921875
0.90625
0.898438
0.898438
0.914063
0.90625
0.914063
0.96875
0.90625
0.929688
0.96875
0.945313
0.945313
0.9375
0.9375
0.929688
0.976563
0.953125
0.953125
0.984375
0.953125
0.960938
0.96875
0.984375
0.953125
0.960938
0.992188
0.976563
0.976563
0.929688
0.960938
0.976563
